## Домашнее задание 6

В данном домашнем задании Вам предстоит обучить нейронную модель лучше ранжировать документы на последней стадии поиска. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1	Переранжирование (1 балл)
### 1.1 Датасет
Для обучения вам предоставляется файл train.tsv в формате аналогичном queries.tsv, используйте эти пары (запрос, документ) в качестве положительных примеров. Отрицательные примеры подберите на своё усмотрение. Используйте полученные в предыдущих домашних работах тексты статей википедии как входные данные для документа.

In [ ]:
from tqdm import tqdm

from typing import Dict, List, Tuple
ArticleName = str
Text = str

In [ ]:
def load_queries(queries_fn: ArticleName) -> List[Tuple[Text, ArticleName]]:
    queries = []
    for line in open(queries_fn, encoding='utf8'):
        query, answer = line.rstrip().split('\t', 1)
        queries.append((query, answer))
    return queries

train = load_queries("/content/drive/MyDrive/HW4/train.tsv")    
for query, article_name in train[:5]:
    print(f'{query} -> {article_name}')

where does the most metabolic activity in the golgi happen -> Cytoplasm
what kind of dog played in turner and hooch -> Dogue_de_Bordeaux
when is there gonna be an eclipse 2017 -> Solar_eclipse_of_August_21,_2017
who won the battle of monitor v. merrimack -> Battle_of_Hampton_Roads
who is competing in big air 2018 olympics -> 2018_Winter_Olympics


In [ ]:
!pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=adf44cdddebd31e092306e43c868e4ecf0f59e6cc6a01872fa2f99035e3b673b
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [ ]:
import gzip
import os
import pandas as pd
import pickle
import re
import requests

from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from hashlib import md5
from tqdm.notebook import tqdm

In [ ]:
directory = "/content/drive/MyDrive/wiki_"
pref = "https://simple.wikipedia.org/"

In [ ]:
agent = UserAgent().chrome

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [ ]:
def hmd5(s):
    return md5(s.encode("utf-8")).hexdigest().lower()

In [ ]:
def make_dirs(path):
    os.makedirs(os.path.dirname(path), exist_ok=True)

In [ ]:
def normalize(parsed):
    parts = []
    for p in parsed.find_all("p"):
        parts.append(re.sub("\[\d*\]|\s+", ' ',p.text).strip())
    lis = set()
    for p in parsed.find_all("span", {"class", "mw-headline"}):
        ul = p.find_next("ul")
        if ul:
            for li in ul.find_all("li"):
                res = li.find("a", {"class":False})
                if res and res.text not in lis and "wiki" not in res.text and res.get("title") and "Special" not in res.get("title"):
                    parts.append(res.text)
                    lis.add(res.text)
    text = " ".join(parts)
    return text

In [ ]:
def get_article_text(article_name: ArticleName) -> Text:
    article_hash = hmd5(article_name)
    dump_path = directory + "/" + article_hash[:2] + "/" + article_hash[2:4] + "/" + article_hash[4:]
    if not os.path.exists(dump_path):
        url = pref + "wiki/" + article_name
        response = requests.get(url, headers={'User-Agent': agent})
        if response.status_code != 404:
            while response.status_code != 200:
                time.sleep(0.1)
                response = requests.get(url, headers={'User-Agent': agent})
                
            make_dirs(dump_path)
            with gzip.open(dump_path, "wb") as f:
                f.write(response.text.encode("utf-8"))
            parsed = bs(response.text, 'html.parser')
            return normalize(parsed)
    else:
        with gzip.open(dump_path, "rb") as f:
            parsed = bs(f.read().decode("utf-8"), "html.parser")
            return normalize(parsed)

In [ ]:
for _, article_name in tqdm(train[:5]):
    print(f'Article: {article_name}\n{get_article_text(article_name)}\n\n')

  0%|          | 0/5 [00:00<?, ?it/s]

Article: Cytoplasm
The cytoplasm (also known as cytosol) is the protoplasm of a cell outside the cell nucleus. It is the jelly-like material plus the organelles outside the nucleus, and inside the cell membrane. Many important functions of a cell take place in organelles, which are like bits of machinery for doing many jobs. The cytoplasm is highly structured: it is not some kind of soup, even though it is made out of 75–80% water. The cytosol contains a complex mixture of cytoskeleton filaments, including the endoplasmic reticulum and the Golgi apparatus. It includes dissolved molecules, and water that fills much of the cell. Due to this network of fibres, membranes and the many dissolved macromolecules such as proteins, the cytosol does not act like a simple liquid. The main function of the cytoplasm is to hold the organelles in place. 


Article: Dogue_de_Bordeaux
The Dogue de Bordeaux, Bordeaux Mastiff, French Mastiff is a dog breed that originated in France. The dogs are large, an

In [ ]:
articles = {article_name for _, article_name in train}

In [ ]:
len(articles)

16732

In [ ]:
from multiprocessing import Pool

def load_docs(articles: List[ArticleName], threads: int = 4) -> Dict[ArticleName, Text]:    
    docs = {}
    pool = Pool(threads)
    tasks = []
    if os.path.exists("/content/drive/MyDrive/HW4/docs"):
        with open("/content/drive/MyDrive/HW4/docs", "rb") as f:
            docs = pickle.load(f)
        return docs

    tasks = []
    for article_name in tqdm(articles):
        tasks.append((article_name, pool.apply_async(get_article_text, (article_name, ))))
    for article_name, task in tqdm(tasks):
        docs[article_name] = task.get(10**6)
    return docs
    
docs = load_docs(articles, 32)
print(f'{len(docs)} docs loaded')

16732 docs loaded


In [ ]:
with open("/content/drive/MyDrive/HW4/docs", "wb") as f:
    pickle.dump(docs, f)

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [ ]:
def remove_stop_words(text):
    tokenizer = RegexpTokenizer(r'\w+|\d+')
    words = tokenizer.tokenize(text)
    ps = nltk.stem.PorterStemmer()
    filtered_words = [ps.stem(word) for word in words if word not in stopwords.words('english') and len(word) > 1]
    return filtered_words

In [ ]:
from typing import Dict, List, Tuple
ArticleName = str
Text = str
Term = str
CollectionData = None
RankingParams = None

In [ ]:
def make_terms(text: Text) -> List[Term]:
    return remove_stop_words(text)

In [ ]:
import numpy as np

In [ ]:
DocId = int
TermFreq = int
RelevInfo = Counter
Posting = List[Tuple[DocId, RelevInfo]]

In [ ]:
import re

In [ ]:
for i, doc in enumerate(docs):
    if docs[doc] is None:
        tokenizer = RegexpTokenizer(r'\w+|\d+')
        content = re.sub('[^a-zA-Z0-9-]', ' ', doc)
        docs[doc] = content

In [ ]:
def invert_index(docs: Dict[ArticleName, Text]) -> CollectionData:
    term_frequency = defaultdict(lambda: Counter())
    doc_id = dict()
    id_doc = dict()
    docs_len = 0
    doc_len = dict()
    
    for idx, doc in tqdm(enumerate(docs)):
        doc_id[doc] = idx
        id_doc[idx] = doc
        if docs[doc] is None:
            doc_terms = Counter(make_terms(re.sub('[^a-zA-Z0-9]', ' ', doc)))
        else:
            doc_terms = Counter(make_terms(docs[doc]))
        
        docs_len += len(doc_terms)
        doc_len[idx] = len(doc_terms)
        for term in doc_terms:
            term_frequency[term][idx] = doc_terms[term]
    
    for term in term_frequency:
        term_frequency[term] = list(sorted(term_frequency[term].items(), key=lambda x: x[0]))

    average_doc_len = docs_len / len(docs)
    
    return {"term_frequency" : term_frequency,
            "doc_id" : doc_id,
            "id_doc" : id_doc,
            "doc_len" : doc_len,
            "average_doc_len" : average_doc_len}

collection_data = invert_index(docs)

0it [00:00, ?it/s]

In [ ]:
index = dict(collection_data["term_frequency"])
del collection_data["term_frequency"]

In [ ]:
def dump_index(index: Dict[Term, Posting], filename: str) -> None:
    with open(filename, "wb") as f:
        for term in sorted(index.keys()):
            pickle.dump((term, index[term]), f)


def dump_collectiondata(data: CollectionData, filename: str) -> None:
    with open(filename, "wb") as f:
        pickle.dump(data, f)

dump_index(index, "/content/drive/MyDrive/HW4/index.inv")


dump_collectiondata(collection_data, "/content/drive/MyDrive/HW4/index.data")


In [ ]:
def load_index(filename: str) -> Dict[Term, Posting]:
    index = dict()
    with open(filename, "rb") as f:
        while True:
            try:
                pl = pickle.load(f)
                index[pl[0]] = pl[1]
            except EOFError:
                break
        return index  

    
def load_collectiondata(filename: str) -> CollectionData:
    with open(filename, "rb") as f:
        return pickle.load(f) 

index = load_index("/content/drive/MyDrive/HW4/index.inv")
print("Number or terms in index:", len(index))
collection_data = load_collectiondata("/content/drive/MyDrive/HW4/index.data")

Number or terms in index: 84926


In [ ]:
import heapq

In [ ]:
def search_indexed(query: Text,
                   top_size: int,
                   index: Dict[Term, Posting],
                   collection_data: CollectionData,
                   ranking_params: RankingParams) -> List[Tuple[ArticleName, float]]:
    result = []

    doc_id = collection_data["doc_id"]
    id_doc = collection_data["id_doc"]
    doc_len = collection_data["doc_len"]
    average_doc_len = collection_data["average_doc_len"]
    
    k1 = ranking_params["k1"]
    k2 = ranking_params["k2"]
    b = ranking_params["b"]    

    f = Counter(make_terms(query))
    terms = f.keys()
    k = len(terms)
    pq = []
    terms = dict(zip(terms, [0] * k))
    cnt = 0
    for term in terms:
        if term in index:
            heapq.heappush(pq, (index[term][0][0], term))
            cnt += 1

    res = []
    while cnt > 0:
        idx = pq[0][0]
        value = 0
        K = k1 * ((1 - b) + b * doc_len[idx] / average_doc_len)

        while len(pq) > 0 and pq[0][0] == idx:
            _, term = heapq.heappop(pq)
            f_ij = index[term][terms[term]][1]
            value += np.log((len(doc_len) + 1) / len(index[term])) *\
                     ((k1 + 1) * f_ij / (K + f_ij)) *\
                     ((k2 + 1) * f[term] / (k2 + f[term]))
            
            terms[term] += 1
            if terms[term] < len(index[term]):
                heapq.heappush(pq, (index[term][terms[term]][0], term))
            else:
                cnt -= 1
        if len(res) < top_size or res[0][0] < value:
            heapq.heappush(res, (value, idx))
        if len(res) > top_size:
            heapq.heappop(res)
    
    res = [(id_doc[idx], value) for value, idx in res]
    res.sort(key=lambda x: x[1], reverse=True)
    return res
ranking_params = {"type": "BM25",
                  "k1": 1,
                  "k2": 1,
                  "b": 1}
for query in ["coronovirus in belarus",
              "who won junior eurovision in 2005",
              "science about full-text search",
             ]:
    result = search_indexed(query, 5, index, collection_data, ranking_params)[:5]
    print(f"[{query}]")
    for article_name, score in result:
        print(f"{score:7.2f}  {article_name}")
    print("\n")

[coronovirus in belarus]
  10.72  Belarus
   9.88  Alexander_Lukashenko
   9.50  Satellite_state
   9.28  Commonwealth_of_Independent_States
   8.71  Minsk


[who won junior eurovision in 2005]
  12.52  Eurovision_Song_Contest_2017
  12.43  Eurovision_Song_Contest
  12.28  Junior_Certificate
  11.38  Eurovision_Song_Contest_2019
  10.77  Eurovision_Song_Contest_2018


[science about full-text search]
  12.58  Google_Search
  12.29  Search_engine
  12.26  Text_editor
  12.01  Help:Starting_a_new_page
  11.08  The_Emoji_Movie




In [ ]:
def load_queries(queries_fn: ArticleName) -> List[Tuple[Text, ArticleName]]:
    queries = []
    for line in open(queries_fn):
        query, answer = line.rstrip().split('\t', 1)
        queries.append((query, answer))
    return queries

queries = load_queries("/content/drive/MyDrive/HW4/train.tsv")
for query, answer in queries:
    assert answer in docs
    
print(f'{len(queries)} queries loaded')
for query, article_name in queries[:5]:
    print(f'{query} -> {article_name}')

50000 queries loaded
where does the most metabolic activity in the golgi happen -> Cytoplasm
what kind of dog played in turner and hooch -> Dogue_de_Bordeaux
when is there gonna be an eclipse 2017 -> Solar_eclipse_of_August_21,_2017
who won the battle of monitor v. merrimack -> Battle_of_Hampton_Roads
who is competing in big air 2018 olympics -> 2018_Winter_Olympics


In [ ]:
def run(title,
        queries: List[Tuple[Text, ArticleName]],
        index: Dict[Term, Posting],
        collection_data: CollectionData,
        ranking_params: RankingParams) -> None:
    accuracy = 0.0
    accuracy10 = 0.0
    rr = 0.0
    processed = 0
    with tqdm(queries) as progress:
        for query, answer in progress:
            result = search_indexed(query, 10, index, collection_data, ranking_params)[:10]
            
            rank = None
            for position, (article_name, score) in enumerate(result):
                if article_name == answer:
                    rank = position + 1
                    break
                
            if rank is not None:
                accuracy += (rank == 1)
                accuracy10 += (rank <= 10)
                rr += 1.0 / rank
                
            processed += 1
            progress.set_description(f'Acc: {accuracy/processed:0.2f}, Acc10: {accuracy10/processed:0.2f}, RR: {rr/processed:0.2f}')
    print(f'{title}\n  Accuracy: {accuracy/processed:0.2f}\n  Accuracy10: {accuracy10/processed:0.2f}\n  RR: {rr/processed:0.2f}')
    
ranking_params = {"type": "BM25",
          "b": 0.871875,
          "k1": 8.0,
          "k2": 0.5}
run("JustRun", queries, index, collection_data, ranking_params)

  0%|          | 0/50000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
def search_pruned(query: Text,
                   top_size: int,
                   pruned_index: Dict[Term, Posting],
                   index: Dict[Term, Posting],
                   collection_data: CollectionData,
                   ranking_params: RankingParams) -> List[Tuple[ArticleName, float]]:
    result = []

    doc_id = collection_data["doc_id"]
    id_doc = collection_data["id_doc"]
    doc_len = collection_data["doc_len"]
    average_doc_len = collection_data["average_doc_len"]
    
    k1 = ranking_params["k1"]
    k2 = ranking_params["k2"]
    b = ranking_params["b"]    

    f = Counter(make_terms(query))
    terms = f.keys()
    k = len(terms)
    pq = []
    terms = dict(zip(terms, [0] * k))
    cnt = 0
    for term in terms:
        if term in pruned_index:
            heapq.heappush(pq, (pruned_index[term][0][0], term))
            cnt += 1

    res = []
    while cnt > 0:
        idx = pq[0][0]
        value = 0
        K = k1 * ((1 - b) + b * doc_len[idx] / average_doc_len)

        while len(pq) > 0 and pq[0][0] == idx:
            _, term = heapq.heappop(pq)
            f_ij = pruned_index[term][terms[term]][1]
            value += np.log((len(doc_len) + 1) / len(index[term])) *\
                     ((k1 + 1) * f_ij / (K + f_ij)) *\
                     ((k2 + 1) * f[term] / (k2 + f[term]))
            
            terms[term] += 1
            if terms[term] < len(pruned_index[term]):
                heapq.heappush(pq, (pruned_index[term][terms[term]][0], term))
            else:
                cnt -= 1
        if len(res) < top_size or res[0][0] < value:
            heapq.heappush(res, (value, idx))
        if len(res) > top_size:
            heapq.heappop(res)
    
    res = [(id_doc[idx], value) for value, idx in res]
    res.sort(key=lambda x: x[1], reverse=True)
    return res

In [ ]:
def run_pruned(title,
        queries: List[Tuple[Text, ArticleName]],
        pruned_index: Dict[Term, Posting],
        index: Dict[Term, Posting],
        collection_data: CollectionData,
        ranking_params: RankingParams) -> None:
    accuracy = 0.0
    accuracy10 = 0.0
    rr = 0.0
    processed = 0
    with tqdm(queries) as progress:
        for query, answer in progress:
            result = search_pruned(query, 10, pruned_index, index, collection_data, ranking_params)[:10]
            
            rank = None
            for position, (article_name, score) in enumerate(result):
                if article_name == answer:
                    rank = position + 1
                    break
                
            if rank is not None:
                accuracy += (rank == 1)
                accuracy10 += (rank <= 10)
                rr += 1.0 / rank
                
            processed += 1
            progress.set_description(f'Acc: {accuracy/processed:0.2f}, Acc10: {accuracy10/processed:0.2f}, RR: {rr/processed:0.2f}')
    print(f'{title}\n  Accuracy: {accuracy/processed:0.2f}\n  Accuracy10: {accuracy10/processed:0.2f}\n  RR: {rr/processed:0.2f}')

In [ ]:
def prune(index: Dict[Term, Posting], top_size: int = 50) -> Dict[Term, Posting]:
    pruned_index = {term : 
                    sorted(sorted(index[term],
                           key=lambda x: x[1],
                           reverse=True)[:top_size],
                           key=lambda x: x[0])
                    for term in index}
    return pruned_index

pruned_index = prune(index, 50)
for term, posting in pruned_index.items():
    prev_doc_id = -1
    for doc_id, freq in posting:
        assert doc_id > prev_doc_id
        prev_doc_id = doc_id

In [ ]:
ranking_params = {"type": "BM25",
          "b": 0.871875,
          "k1": 8.0,
          "k2": 0.5}
for query in ["coronovirus in belarus",
              "who won junior eurovision in 2005",
              "science about full-text search",
             ]:
    result = search_pruned(query, 5, pruned_index, index, collection_data, ranking_params)[:5]
    print(f"[{query}]")
    for article_name, score in result:
        print(f"{score:7.2f}  {article_name}")
    print("\n")
run_pruned("JustRun", queries, pruned_index, index, collection_data, ranking_params)

[coronovirus in belarus]
  27.02  Belarus
  18.17  Alexander_Lukashenko
  15.72  Commonwealth_of_Independent_States
  13.60  Satellite_state
  13.44  Minsk


[who won junior eurovision in 2005]
  25.48  Eurovision_Song_Contest
  19.26  Middle_school
  18.74  Eurovision_Song_Contest_2019
  18.34  Puppy_Dog_Pals
  16.37  Meryl_Davis


[science about full-text search]
  30.64  Google_Search
  28.45  Search_engine
  27.16  Search_algorithm
  26.78  Text_editor
  24.05  Search_engine_optimization




  0%|          | 0/50000 [00:00<?, ?it/s]

JustRun
  Accuracy: 0.24
  Accuracy10: 0.52
  RR: 0.33


### 1.2. Обучение
Используйте готовую предобученную BERT-оподобную модель в качестве основы. Дообучите её на задачу бинарной классификации - предсказания того, является ли документ релевантным запросу.
Рекомендую пользоваться библиотекой transformers ([документация](https://huggingface.co/transformers/)). Она поддерживает pytorch и tensorflow, содержит репозиторий предобученных моделей.
Рекомендую использовать colab с настройками среды GPU, чтобы ускорить обучение и применение.


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 59 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
     |████████████████████████████████| 3.3 MB 24.1 MB/s 
     |████████████████████████████████| 895 kB 53.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
batch = [["the most common wave forms of ac is", "Waveform"],
         ["how many seats did labour win in 2015", "2015 United Kingdom general election"],
         ["who won the soccer game monterrey or tigres", "Tigres UANL"],
         ["capital and largest city of belarus on the svislach river", "Minsk"]]
encoded_input = tokenizer(batch, padding=True, return_tensors='pt')
for ids in encoded_input['input_ids']:
  print(ids)
  print(tokenizer.convert_ids_to_tokens(ids))
  print('\n')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

tensor([  101,  1996,  2087,  2691,  4400,  3596,  1997,  9353,  2003,   102,
         4400, 14192,   102,     0,     0,     0,     0,     0])
['[CLS]', 'the', 'most', 'common', 'wave', 'forms', 'of', 'ac', 'is', '[SEP]', 'wave', '##form', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


tensor([ 101, 2129, 2116, 4272, 2106, 4428, 2663, 1999, 2325,  102, 2325, 2142,
        2983, 2236, 2602,  102,    0,    0])
['[CLS]', 'how', 'many', 'seats', 'did', 'labour', 'win', 'in', '2015', '[SEP]', '2015', 'united', 'kingdom', 'general', 'election', '[SEP]', '[PAD]', '[PAD]']


tensor([  101,  2040,  2180,  1996,  4715,  2208, 26843,  2030, 14841, 17603,
         2015,   102, 14841, 17603,  2015, 25423, 20554,   102])
['[CLS]', 'who', 'won', 'the', 'soccer', 'game', 'monterrey', 'or', 'ti', '##gre', '##s', '[SEP]', 'ti', '##gre', '##s', 'ua', '##nl', '[SEP]']


tensor([  101,  3007,  1998,  2922,  2103,  1997, 12545,  2006,  1996, 17917,
         2483,  2721,  2818,  2314,   102, 20790, 

In [ ]:
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
import torch

In [ ]:
import numpy as np

In [ ]:
class QueriesDataset:
    def __init__(self, data, top_size):
        self.data = data
        self.top_size = top_size
        self.queries = np.array(list(data.keys()))
        self.articles = np.array([data[query] for query in data.keys()])

    def __len__(self):
        return len(self.queries) * 2 * (self.top_size - 1)
    
    def __getitem__(self, i):
        q_id = i // (2 * (self.top_size - 1))
        a_id = i % (2 * (self.top_size - 1))
        if a_id < self.top_size - 1:
        #     return [self.queries[q_id], re.sub('_', ' ', self.articles[q_id][0])], 1.0
        # else:
        #     return [self.queries[q_id], re.sub('_', ' ', self.articles[q_id - self.top_size + 2][0])], 0.0
            return [self.queries[q_id], ' '.join(docs[self.articles[q_id][0]].split()[:20])], 1.0
        else:
            return [self.queries[q_id], ' '.join(docs[self.articles[q_id - self.top_size + 2][0]].split()[:20])], 0.0

In [ ]:
top_size = 5
for _, article_name in tqdm(bad_queries):
    result = search_indexed(docs[article_name], 5, index, collection_data, ranking_params)
    data = [article_name]
    for article,_ in result:
        if article != article_name and len(data) < top_size:
            data.append(article)
    if len(data) < top_size:
        print(query, article_name, len(data))

In [ ]:
def get_dataset(queries: List[Tuple[Text, ArticleName]],
                top_size: int,
                index: Dict[Term, Posting],
                collection_data: CollectionData,
                ranking_params: RankingParams):
    dataset = dict()
    for query, article_name in tqdm(queries):
        # result = search_indexed(query, top_size, index, collection_data, ranking_params)
        result = search_pruned(query, 5, pruned_index, index, collection_data, ranking_params)
        data = [article_name]
        for article,_ in result:
            if article != article_name and len(data) < top_size:
                data.append(article)
        if len(data) < top_size:
            result = search_pruned(query + ' ' + docs[article_name], 5, pruned_index, index, collection_data, ranking_params)
            data = [article_name]
            for article,_ in result:
                if article != article_name and len(data) < top_size:
                    data.append(article)
        dataset[query] = data
    return QueriesDataset(dataset, top_size)
    # return bad_queries

In [ ]:
def get_dataset(queries: List[Tuple[Text, ArticleName]],
                top_size: int,
                index: Dict[Term, Posting],
                collection_data: CollectionData,
                ranking_params: RankingParams):
    dataset = dict()
    for query, article_name in tqdm(queries):
        # result = search_indexed(query, top_size, index, collection_data, ranking_params)
        result = search_pruned(query, 5, pruned_index, index, collection_data, ranking_params)
        if len(result) < 5:
            print("Query:", query, "--", "Article: ", article_name)
        # data = [article_name]
        # for article,_ in result:
        #     if article != article_name and len(data) < top_size:
        #         data.append(article)
        # if len(data) < top_size:
        #     result = search_pruned(query + ' ' + docs[article_name], 5, pruned_index, index, collection_data, ranking_params)
        #     data = [article_name]
        #     for article,_ in result:
        #         if article != article_name and len(data) < top_size:
        #             data.append(article)
        # dataset[query] = data
    return QueriesDataset(dataset, top_size)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, val = train_test_split(train, random_state=42, test_size=0.2)
val, test = train_test_split(val, random_state=42, test_size=0.5)

In [ ]:
from transformers import AutoModel, AdamW

In [ ]:
model = AutoModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch.nn as nn

In [ ]:
class BertForRanking(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.tail = nn.Sequential(
            nn.Linear(768, 256),
            nn.BatchNorm1d(256),
            nn.Mish(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.Mish(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        inputs, attention = x
        return self.tail(self.model(inputs, attention)[1])[:,0]

In [ ]:
get_dataset(train,
            5,
            index, 
            collection_data,
            ranking_params)

  0%|          | 0/40000 [00:00<?, ?it/s]

Query: r e l u c t a n c e -- Article:  Magnetic_reluctance
Query: s n d o m i n g o d r -- Article:  Santo_Domingo
Query: what is there to do in bluefield wv -- Article:  Bluefield,_West_Virginia
Query: who are the socs and who are the greasers -- Article:  The_Outsiders_(novel)
Query: h e p t a t h l o n -- Article:  Heptathlon
Query: s i l d e n a f i l -- Article:  Sildenafil
Query: is this the mpla or is this the uda -- Article:  Anarchy_in_the_U.K.
Query: a p p l i c a t i o n -- Article:  Application
Query: what is wifi b/g/n -- Article:  IEEE_802.11
Query: i n n o c e n c e -- Article:  Innocence


KeyboardInterrupt: ignored

In [ ]:
ranking_params = {"type": "BM25",
          "b": 0.871875,
          "k1": 8.0,
          "k2": 0.5}
train_dataset = get_dataset(train,
                            5,
                            index,
                            collection_data,
                            ranking_params)
val_dataset = get_dataset(val,
                          5,
                          index,
                          collection_data,
                          ranking_params)

  0%|          | 0/40000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def collate_fn(data):
    res = tokenizer([d[0] for d in data], padding=True, return_tensors='pt')
    return (res['input_ids'], res['attention_mask']), torch.tensor([d[1] for d in data], dtype=torch.float32)

In [ ]:
BATCH_SIZE = 16
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=collate_fn,
                                           num_workers=2, 
                                           pin_memory=True,
                                           shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=collate_fn,
                                           num_workers=2, 
                                           pin_memory=True)

In [ ]:
train_dataset[0]

(['where was the first car invented in the world',
  'The early history of the automobile can be divided into a number of eras, based on the method of propulsion.'],
 1.0)

In [ ]:
print(f"Length train: {len(train_loader)}")
print(f"Length valid: {len(valid_loader)}")

Length train: 20000
Length valid: 2500


In [ ]:
bert = BertForRanking(model).to(device)

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)


In [ ]:
device

device(type='cuda', index=0)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
num_epochs = 2
n_log = 1000
cur_loss = 0.0
acc = 0.0
best_loss = None
# with tqdm(range(num_epochs)) as progress:
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    bert.train()
    with tqdm(enumerate(train_loader)) as progress:
        for i, data in progress:
            inputs, labels = data
            inputs = (inputs[0].to(device), inputs[1].to(device))
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = bert(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            acc += ((outputs > 0) == labels).float().sum().item()
            cur_loss += loss.item()
            progress.set_description(f'Train Acc: {acc/(i + 1) / 16:0.5f}, Train loss: {cur_loss/(i + 1) / 16:0.5f}')

    cur_loss = 0.0
    acc = 0.0
    
    bert.eval()
    with torch.no_grad():
        for data in tqdm(valid_loader):
            inputs, labels = data
            inputs = (inputs[0].to(device), inputs[1].to(device))
            labels = labels.to(device)
            # optimizer.zero_grad()
            outputs = bert(inputs)
            loss = criterion(outputs, labels)
            acc += ((outputs > 0) == labels).float().sum().item()
            cur_loss += loss.item()
        print(cur_loss / len(valid_loader))
        print(acc / len(valid_loader) / 16)
    if best_loss is None or cur_loss < best_loss:
        torch.save(bert.state_dict(), 'model.pt')
        best_loss = running_loss    

0it [00:00, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.save(bert.state_dict(), 'model.pt')

In [ ]:
test_dataset = get_dataset(val,
                          5,
                          index,
                          collection_data,
                          ranking_params)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=64, 
                                           collate_fn=collate_fn,
                                           num_workers=2, 
                                           pin_memory=True)

In [ ]:
bert.load_state_dict(torch.load('/content/drive/MyDrive/HW4/model.pt'))

In [ ]:
bert.eval()
with torch.no_grad():
    running_loss = 0.0
    running_acc = 0.0
    for data in tqdm(test_loader):
        inputs, labels = data
        inputs = (inputs[0].to(device), inputs[1].to(device))
        labels = labels.to(device)
        # optimizer.zero_grad()
        outputs = bert(inputs)
        loss = criterion(outputs, labels)
        running_acc += ((outputs > 0) == labels).float().sum().item()
        running_loss += loss.item()
    print(running_loss / len(test_loader))
    print(running_acc / len(test_loader) / 64)
    # writer.add_scalar('val_loss', running_loss / len(valid_loader), epoch)
    # writer.add_scalar('val_acc', running_acc / len(valid_loader), epoch)

  0%|          | 0/625 [00:00<?, ?it/s]

0.124499815762043
0.9696


In [ ]:
def search_bert(query: Text,
                top_size: int,
                index: Dict[Term, Posting],
                collection_data: CollectionData,
                ranking_params: RankingParams) -> List[Tuple[ArticleName, float]]:
    articles = search_indexed(query, top_size, index, collection_data, ranking_params)
    if len(articles) == 0:
        articles = search_bert(''.join(query.split()), top_size, index, collection_data, ranking_params)
    # result = [[query, re.sub('_', ' ', article_name)] for article_name,_ in articles]
    result = [[query, ' '.join(docs[article_name].split()[:20])] for article_name,_ in articles]
    inputs = tokenizer(result, padding=True, return_tensors='pt')
    pred = bert((inputs['input_ids'].to(device), inputs['attention_mask'].to(device)))
    pred = [(articles[i][0], score) for i, score in enumerate(pred.detach().cpu().numpy())]
    res = sorted(pred, key=lambda x: x[1], reverse=True)
    return res

In [ ]:
queries[1]

('what kind of dog played in turner and hooch', 'Dogue_de_Bordeaux')

In [ ]:
search_bert(queries[0][0], 5, index, collection_data, ranking_params)

[('Golgi_complex', 3.0193295),
 ('Metabolic_rate', 3.0186942),
 ('Catabolism', 2.9395354),
 ('Vitamin_B6', -1.2295434),
 ('Functional_magnetic_resonance_imaging', -2.823289)]

In [ ]:
search_bert(queries[1][0], 5, index, collection_data, ranking_params)

[('Staffordshire_Bull_Terrier', 3.2380466),
 ('Hot_dog', 3.2202024),
 ('Briard', 3.191447),
 ('Dachshund', 3.1154687),
 ('Sylvester_Turner', 3.1132085)]

In [ ]:
for query in ["coronovirus in belarus",
              "who won junior eurovision in 2005",
              "science about full-text search",
             ]:
    result = search_bert(query, 5, index, collection_data, ranking_params)[:5]
    print(f"[{query}]")
    for article_name, score in result:
        print(f"{score:7.2f}  {article_name}")
    print("\n")
# run_pruned("JustRun", queries, pruned_index, index, collection_data, ranking_params)

[coronovirus in belarus]
   3.20  Minsk
   3.12  Belarus
   2.15  Commonwealth_of_Independent_States
   0.55  Alexander_Lukashenko
  -1.89  Satellite_state


[who won junior eurovision in 2005]
   3.12  Eurovision_Song_Contest
   2.99  Eurovision_Song_Contest_2019
   2.32  Junior_Certificate
  -2.46  Puppy_Dog_Pals
  -2.54  Middle_school


[science about full-text search]
   3.25  Search_engine_optimization
   3.24  Search_algorithm
   3.22  Google_Search
   3.14  Text_editor
   3.08  Search_engine




In [ ]:
for query in ["coronovirus in belarus",
              "who won junior eurovision in 2005",
              "science about full-text search",
             ]:
    result = search_bert(query, 5, index, collection_data, ranking_params)[:5]
    print(f"[{query}]")
    for article_name, score in result:
        print(f"{score:7.2f}  {article_name}")
    print("\n")
# run_pruned("JustRun", queries, pruned_index, index, collection_data, ranking_params)

[coronovirus in belarus]
   3.03  Belarus
   2.87  Minsk
   2.38  Alexander_Lukashenko
  -2.51  Commonwealth_of_Independent_States
  -2.57  Satellite_state


[who won junior eurovision in 2005]
   2.94  Eurovision_Song_Contest_2019
   2.77  Eurovision_Song_Contest
  -2.80  Junior_Certificate
  -2.82  Middle_school
  -2.83  Puppy_Dog_Pals


[science about full-text search]
   2.91  Search_algorithm
   2.85  Text_editor
   2.75  Search_engine
   2.63  Search_engine_optimization
   1.39  Google_Search




In [ ]:
def run_bert(title,
        queries: List[Tuple[Text, ArticleName]],
        top_size: int,
        index: Dict[Term, Posting],
        collection_data: CollectionData,
        ranking_params: RankingParams) -> None:
    accuracy = 0.0
    accuracy10 = 0.0
    rr = 0.0
    processed = 0
    with tqdm(queries) as progress:
        for query, answer in progress:
            result = search_bert(query, top_size, index, collection_data, ranking_params)
            rank = None
            for position, (article_name, score) in enumerate(result):
                if article_name == answer:
                    rank = position + 1
                    break
                
            if rank is not None:
                accuracy += (rank == 1)
                accuracy10 += (rank <= top_size)
                rr += 1.0 / rank
                
            processed += 1
            progress.set_description(f'Acc: {accuracy/processed:0.2f}, Acc10: {accuracy10/processed:0.2f}, RR: {rr/processed:0.2f}')
    print(f'{title}\n  Accuracy: {accuracy/processed:0.2f}\n  Accuracy10: {accuracy10/processed:0.2f}\n  RR: {rr/processed:0.2f}')
    
ranking_params = {"type": "BM25",
          "b": 0.871875,
          "k1": 8.0,
          "k2": 0.5}
run_bert("JustRun", test, 5, index, collection_data, ranking_params)

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
def run_bert(title,
        queries: List[Tuple[Text, ArticleName]],
        top_size: int,
        index: Dict[Term, Posting],
        collection_data: CollectionData,
        ranking_params: RankingParams) -> None:
    accuracy = 0.0
    accuracy10 = 0.0
    rr = 0.0
    processed = 0
    with tqdm(queries) as progress:
        for query, answer in progress:
            result = search_bert(query, top_size, index, collection_data, ranking_params)
            rank = None
            for position, (article_name, score) in enumerate(result):
                if article_name == answer:
                    rank = position + 1
                    break
                
            if rank is not None:
                accuracy += (rank == 1)
                accuracy10 += (rank <= top_size)
                rr += 1.0 / rank
                
            processed += 1
            progress.set_description(f'Acc: {accuracy/processed:0.2f}, Acc10: {accuracy10/processed:0.2f}, RR: {rr/processed:0.2f}')
    print(f'{title}\n  Accuracy: {accuracy/processed:0.2f}\n  Accuracy10: {accuracy10/processed:0.2f}\n  RR: {rr/processed:0.2f}')
    
ranking_params = {"type": "BM25",
          "b": 0.871875,
          "k1": 8.0,
          "k2": 0.5}
run_bert("JustRun", test, 5, index, collection_data, ranking_params)

  0%|          | 0/5000 [00:00<?, ?it/s]

JustRun
  Accuracy: 0.37
  Accuracy10: 0.56
  RR: 0.45


In [ ]:
for i in [10, 25, 50]:
    run_bert("JustRun", test, i, index, collection_data, ranking_params)

  0%|          | 0/5000 [00:00<?, ?it/s]

### 1.3. Поиск
Используйте предсказания модели для переранжирования лучших N документов, полученных при поиске BM25. Сравните качество полученного алгоритма поиска при разных значения N (10, 25, 50).


In [ ]:
#
#
#
#
#  CODE HERE
#
#
#
#
#

## 2. Оптимизация применения (1 балл)
Обучите BERT-опободную нейронную модель в [сиамской схеме](https://www.researchgate.net/profile/Laure_Soulier/publication/304350730/figure/download/fig1/AS:376342722957320@1466738505636/General-architecture-of-the-DSSM-network.png), аналогичной DSSM, когда запросная и документная часть считаются независимо, и связываются только в конце через косинус.
Предпрочитайте векторные представления для запросов и документов, требуемых для оценки качества.
Сравните качество и **скорость** полученной модели с предыдущим пунктом.

In [ ]:
#
#
#
#
#  CODE HERE
#
#
#
#
#